# Baryon->Baryon Correlator

First we import stuff, and define all of the objects needed to create/annihilate baryons.  In the future this will be read in from a text file that is generated by some group theory code to construct operators in an automated way.  

In [1]:
import sys
import os

sys.path.append(os.getcwd()[:-9])

from WickContractions.ops.operator import *
from WickContractions.ops.elemental import *
from WickContractions.ops.quarks import *
from WickContractions.ops.commuting import *
from WickContractions.corrs.diagram import *
from WickContractions.wick.utilities import *
from WickContractions.wick.contract import *
from WickContractions.laph.diagram import *

from Nc_baryon import get_operators

## Run operator construction code

In [2]:
operator = get_operators({'nc': 4, 
                     'I': 2, 'Iz': 2,
                     'S': 2, 'Sz': 2}
                    ,'4Weyl')


Nc = 4
 I = 2.0
Iz = 2.0
 S = 2.0
Sz = 2.0

Checking eigenvalues of I^2 and S^2 ...

I and S okay
good


Checking eigenvalues of I^2 and S^2 subspaces
 ...
I and S okay
good

NOTE that in order to make symmetric wavefunction,
 can only combine following isospin and spin eigenvalues
I*(I+1) = 6.0, S*(S+1) = 6.0


Basis
-----
v1: uu,uu,uu,uu type (sum over 1 permutations)

Solution
--------
I^2 eigenvalue: 6.0
I^2 eigen vector
[1.]

S^2 eigenvalue: 6.0
S^2 eigen vector
[1.]

In gamma workflow, gamma matrix shape is
(16, 256)
Saving matrices to files.


In [3]:
for k,v in operator.items():
    print(k)
    print(v['factor'])

0000
[1.0]


## Wick Contract the operators

In [4]:
annihilation_elementals=[]

intToUD = {'0': 'u', '1': 'd'}

for k,v in operator.items():

    c0=EpsilonTensor(['c0','c1','c2','c3'])
    c1=SpinMatrix('X1^'+k,['s0','s1','s2','s3'])
    
    
    q0=Quark(False,intToUD[k[0]],'s0','c0','tf','x1')
    q1=Quark(False,intToUD[k[1]],'s1','c1','tf','x1')
    q2=Quark(False,intToUD[k[2]],'s2','c2','tf','x1')
    q3=Quark(False,intToUD[k[3]],'s3','c3','tf','x1')

    annihilation_elementals.append(ElementalOperator(v['factor'][0],[c0,c1],[q0,q1,q2,q3]))
    
annihilate_baryon = Operator(annihilation_elementals)

creation_elementals = []
for k,v in operator.items():
    c2=EpsilonTensor(['c4','c5','c6','c7'])
    c3=SpinMatrix('X2^'+k,['s4','s5','s6','s7'])

    q4=Quark(True,intToUD[k[0]],'s4','c4','ti','x2')
    q5=Quark(True,intToUD[k[1]],'s5','c5','ti','x2')
    q6=Quark(True,intToUD[k[2]],'s6','c6','ti','x2')
    q7=Quark(True,intToUD[k[3]],'s7','c7','ti','x2')
    
    creation_elementals.append(ElementalOperator(v['factor'][0],[c2,c3],[q4,q5,q6,q7]))

create_baryon = Operator(creation_elementals)

In [5]:
print(annihilate_baryon)

1.0*eps_{c0 c1 c2 c3}*X1^0000_{s0 s1 s2 s3}*u_{s0 c0}(x1, tf)*u_{s1 c1}(x1, tf)*u_{s2 c2}(x1, tf)*u_{s3 c3}(x1, tf)


In [6]:
print(create_baryon)

1.0*eps_{c4 c5 c6 c7}*X2^0000_{s4 s5 s6 s7}*\bar{u}_{s4 c4}(x2, ti)*\bar{u}_{s5 c5}(x2, ti)*\bar{u}_{s6 c6}(x2, ti)*\bar{u}_{s7 c7}(x2, ti)


Now do the contractions and store in Laph subspace

In [7]:
res = contract(annihilate_baryon, create_baryon)

for i in range(len(res.diagrams)):
    res.diagrams[i]=LDiagram(res.diagrams[i])
    

In [8]:
print("B->B correlator has {} diagrams".format(len(res.diagrams)))

B->B correlator has 24 diagrams


Now we can apply laph smearing to the quarks.  We print out the B->B diagrams explicitly since there are only four. 

## Diagrams in terms of baryon blocks

In [9]:
#testing a more generic way to make blocks.
tst=copy.deepcopy(res.diagrams)
for d in tst:
    d.create_baryon_blocks()
    #d.create_block('delta','M') # should allow this code to handle any number of mesons/baryons
    d.short_props()
    
for d in tst:
    d.create_baryon_source()       # not NC agnostic at the moment.
                                   # not generalized to accomodate mesons + baryons
    #if rank4tensor is symmetric_pairwise:
    #    d.use_symmetric_block_properties()
    #else if rank4tensor is antisymmetric_pairwise:
    #    d.use_antisymmetric_block_properties()
    #else:
    #    #hard computation - too bad
    
    print(d)

1.0 B*(X1^0000,tf,ti)_{4 5 6 7}B(X2^0000,ti)_{4 5 6 7}
-1.0 B*(X1^0000,tf,ti)_{5 4 6 7}B(X2^0000,ti)_{4 5 6 7}
1.0 B*(X1^0000,tf,ti)_{5 6 4 7}B(X2^0000,ti)_{4 5 6 7}
-1.0 B*(X1^0000,tf,ti)_{4 6 5 7}B(X2^0000,ti)_{4 5 6 7}
1.0 B*(X1^0000,tf,ti)_{6 4 5 7}B(X2^0000,ti)_{4 5 6 7}
-1.0 B*(X1^0000,tf,ti)_{6 5 4 7}B(X2^0000,ti)_{4 5 6 7}
1.0 B*(X1^0000,tf,ti)_{6 5 7 4}B(X2^0000,ti)_{4 5 6 7}
-1.0 B*(X1^0000,tf,ti)_{6 4 7 5}B(X2^0000,ti)_{4 5 6 7}
1.0 B*(X1^0000,tf,ti)_{4 6 7 5}B(X2^0000,ti)_{4 5 6 7}
-1.0 B*(X1^0000,tf,ti)_{5 6 7 4}B(X2^0000,ti)_{4 5 6 7}
1.0 B*(X1^0000,tf,ti)_{5 4 7 6}B(X2^0000,ti)_{4 5 6 7}
-1.0 B*(X1^0000,tf,ti)_{4 5 7 6}B(X2^0000,ti)_{4 5 6 7}
1.0 B*(X1^0000,tf,ti)_{4 7 5 6}B(X2^0000,ti)_{4 5 6 7}
-1.0 B*(X1^0000,tf,ti)_{5 7 4 6}B(X2^0000,ti)_{4 5 6 7}
1.0 B*(X1^0000,tf,ti)_{5 7 6 4}B(X2^0000,ti)_{4 5 6 7}
-1.0 B*(X1^0000,tf,ti)_{4 7 6 5}B(X2^0000,ti)_{4 5 6 7}
1.0 B*(X1^0000,tf,ti)_{6 7 4 5}B(X2^0000,ti)_{4 5 6 7}
-1.0 B*(X1^0000,tf,ti)_{6 7 5 4}B(X2^0000,ti)_{4 5 6 7}
1

In [10]:
unique_blocks = []

for d in tst:
    for elem in d.commuting:
        name = str(elem).split('_')[0]
        if name not in unique_blocks:
            unique_blocks.append(name)

print(unique_blocks)

['B*(X1^0000,tf,ti)', 'B(X2^0000,ti)']


### Outputting info for computation

I just did a quick check of the loop cost and printed out each contraction that needed to be done.  The latter notation is used in Eigen::Tensor or Fastor.

In [11]:
#brute force counting.
costs = {}
for d in tst:
    nested_sums = 0
    for idx in d.commuting[0].indices[0:4]:
        if(idx in d.commuting[1].indices):
            nested_sums+=1
    if nested_sums in costs:
        costs[nested_sums]+=1
    else:
        costs[nested_sums]=1
print('total loops:')
for k,v in costs.items():
    print('  {}*N^{}'.format(v,k))

total loops:
  24*N^4


In [12]:
contractions = []
for d in tst:
    diag=[]
    for i,idx in enumerate(d.commuting[0].indices[0:4]):
        j=d.commuting[1].indices.index(idx)
        diag.append([i,j])
                    
    b0 = unique_blocks.index( str(d.commuting[0]).split('_')[0] ) 
    b1 = unique_blocks.index( str(d.commuting[1]).split('_')[0] )
    
    contractions.append({'blocks': [b0,b1], 'indices': diag})
print(contractions)

[{'blocks': [0, 1], 'indices': [[0, 0], [1, 1], [2, 2], [3, 3]]}, {'blocks': [0, 1], 'indices': [[0, 1], [1, 0], [2, 2], [3, 3]]}, {'blocks': [0, 1], 'indices': [[0, 1], [1, 2], [2, 0], [3, 3]]}, {'blocks': [0, 1], 'indices': [[0, 0], [1, 2], [2, 1], [3, 3]]}, {'blocks': [0, 1], 'indices': [[0, 2], [1, 0], [2, 1], [3, 3]]}, {'blocks': [0, 1], 'indices': [[0, 2], [1, 1], [2, 0], [3, 3]]}, {'blocks': [0, 1], 'indices': [[0, 2], [1, 1], [2, 3], [3, 0]]}, {'blocks': [0, 1], 'indices': [[0, 2], [1, 0], [2, 3], [3, 1]]}, {'blocks': [0, 1], 'indices': [[0, 0], [1, 2], [2, 3], [3, 1]]}, {'blocks': [0, 1], 'indices': [[0, 1], [1, 2], [2, 3], [3, 0]]}, {'blocks': [0, 1], 'indices': [[0, 1], [1, 0], [2, 3], [3, 2]]}, {'blocks': [0, 1], 'indices': [[0, 0], [1, 1], [2, 3], [3, 2]]}, {'blocks': [0, 1], 'indices': [[0, 0], [1, 3], [2, 1], [3, 2]]}, {'blocks': [0, 1], 'indices': [[0, 1], [1, 3], [2, 0], [3, 2]]}, {'blocks': [0, 1], 'indices': [[0, 1], [1, 3], [2, 2], [3, 0]]}, {'blocks': [0, 1], 'indi

In [13]:
for d in contractions:
    s='diagrams.push_back({},{},std::vector<std::vector<int>>{{'.format(d['blocks'][0],d['blocks'][1])
    for pair in d['indices'][:-1]:
        s+='{{{},{}}},'.format(pair[0],pair[1])
    s+='{{{},{}}}'.format(d['indices'][-1][0],d['indices'][-1][1])
    s+='});'
    print(s)

diagrams.push_back(0,1,std::vector<std::vector<int>>{{0,0},{1,1},{2,2},{3,3}});
diagrams.push_back(0,1,std::vector<std::vector<int>>{{0,1},{1,0},{2,2},{3,3}});
diagrams.push_back(0,1,std::vector<std::vector<int>>{{0,1},{1,2},{2,0},{3,3}});
diagrams.push_back(0,1,std::vector<std::vector<int>>{{0,0},{1,2},{2,1},{3,3}});
diagrams.push_back(0,1,std::vector<std::vector<int>>{{0,2},{1,0},{2,1},{3,3}});
diagrams.push_back(0,1,std::vector<std::vector<int>>{{0,2},{1,1},{2,0},{3,3}});
diagrams.push_back(0,1,std::vector<std::vector<int>>{{0,2},{1,1},{2,3},{3,0}});
diagrams.push_back(0,1,std::vector<std::vector<int>>{{0,2},{1,0},{2,3},{3,1}});
diagrams.push_back(0,1,std::vector<std::vector<int>>{{0,0},{1,2},{2,3},{3,1}});
diagrams.push_back(0,1,std::vector<std::vector<int>>{{0,1},{1,2},{2,3},{3,0}});
diagrams.push_back(0,1,std::vector<std::vector<int>>{{0,1},{1,0},{2,3},{3,2}});
diagrams.push_back(0,1,std::vector<std::vector<int>>{{0,0},{1,1},{2,3},{3,2}});
diagrams.push_back(0,1,std::vector<std::

In [14]:
s='std::vector<double> coef {'
for d in tst[:-1]:
    s+='{},'.format(d.coef)
s+='{}}};'.format(tst[-1].coef)
print(s)

std::vector<double> coef {1.0,-1.0,1.0,-1.0,1.0,-1.0,1.0,-1.0,1.0,-1.0,1.0,-1.0,1.0,-1.0,1.0,-1.0,1.0,-1.0,1.0,-1.0,1.0,-1.0,1.0,-1.0};
